# **В этом блокноте я решаю классическую задачу Kaggle по определению выживших на Титанике, пробую разные модели и методы**

In [380]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [ ]:
#!kaggle competitions download -c titanic

In [382]:
train_dataframe = pd.read_csv('/kaggle/input/titanic/train.csv')
test_dataframe = pd.read_csv('/kaggle/input/titanic/test.csv')
train_dataframe.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


**Проанализируем датасет. Посмотрим на null и подумаем, что с ним можно сделать. Также поглядим на значения, которые могут принимать разные фичи.**

In [383]:
train_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [384]:
train_dataframe['Embarked'].value_counts()

Embarked
S    644
C    168
Q     77
Name: count, dtype: int64

In [385]:
a = {}
for s in train_dataframe.loc[train_dataframe['Cabin'].notna(), 'Cabin']:
    if s[0] in a:
        a[s[0]] += 1
    else:
        a[s[0]] = 1
    if s[0] in 'ABCDEFGT' and a[s[0]] < 5:
        print(s)
a

C85
C123
E46
G6
C103
D56
A6
C23 C25 C27
B78
D33
B30
B28
F33
F G73
E31
A5
D10 D12
D26
B58 B60
E101
F E69
F2
E33
A7
A32
G6
G6
T
G6


{'C': 59, 'E': 32, 'G': 4, 'D': 33, 'A': 15, 'B': 47, 'F': 13, 'T': 1}

In [386]:
test_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [387]:
mean_age = train_dataframe['Age'].mean()
mean_fare = train_dataframe['Fare'].mean()
mean_age, mean_fare

(29.69911764705882, 32.204207968574636)

**Итого, для Cabin я введу обозначения для null: Not stated. Также Cabin я разделю на Letter и Number. Для age и fare я заполню пустые значения средним по train. male/female я превращу в 1/0. Pclass несет в себе категориальный смысл, поэтому его следует пометить как object. Вдобавок ко всему, я нормализую данные. Вуаля, датасет готов!**

In [388]:
def get_first_num(s):
    num = 0
    for letter in s:
        if letter in '0123456789':
            num = num * 10 + int(letter)
        elif num != 0:
            break
    return num

def normalize(ser):
    return (ser - ser.mean()) / ser.std()

def clean_dataframe(dataframe):
    dataframe.loc[dataframe['Cabin'].isna(), 'Cabin'] = 'Not stated'
    dataframe.loc[dataframe['Age'].isna(), 'Age'] = mean_age
    dataframe.loc[dataframe['Fare'].isna(), 'Fare'] = mean_fare
    dataframe['Sex'] = (dataframe['Sex'] == 'male') * 1
    cabin_letter = [s[0] for s in dataframe['Cabin']]
    dataframe['Cabin_Letter'] = cabin_letter
    cabin_number = [get_first_num(s) for s in dataframe['Cabin']]
    dataframe['Cabin_Num'] = cabin_number
    dataframe['Pclass'] = dataframe['Pclass'].astype('object')
    
    dataframe['Age'] = normalize(dataframe['Age'])
    dataframe['Fare'] = normalize(dataframe['Fare'])
    dataframe['Cabin_Num'] = normalize(dataframe['Cabin_Num'])
    
    return dataframe.drop(['Name', 'Ticket', 'Cabin'], axis=1)

In [389]:
train_dataframe['Pclass'].value_counts()

Pclass
3    491
1    216
2    184
Name: count, dtype: int64

In [390]:
train_dataframe = clean_dataframe(train_dataframe)
test_dataframe = clean_dataframe(test_dataframe)
train_dataframe.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Cabin_Letter,Cabin_Num
0,1,0,3,1,-0.592148,1,0,-0.502163,S,N,-0.421085
1,2,1,1,0,0.638430,1,0,0.786404,C,C,2.737053
2,3,1,3,0,-0.284503,0,0,-0.488580,S,N,-0.421085
3,4,1,1,0,0.407697,1,0,0.420494,S,C,4.148926
4,5,0,3,1,0.407697,0,0,-0.486064,S,N,-0.421085


In [391]:
train_dataframe = train_dataframe.dropna()
train_dataframe.info(), test_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Index: 889 entries, 0 to 890
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   889 non-null    int64  
 1   Survived      889 non-null    int64  
 2   Pclass        889 non-null    object 
 3   Sex           889 non-null    int64  
 4   Age           889 non-null    float64
 5   SibSp         889 non-null    int64  
 6   Parch         889 non-null    int64  
 7   Fare          889 non-null    float64
 8   Embarked      889 non-null    object 
 9   Cabin_Letter  889 non-null    object 
 10  Cabin_Num     889 non-null    float64
dtypes: float64(3), int64(5), object(3)
memory usage: 83.3+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   418 non-null    int64  
 1   Pclass        418 non-null    object 
 

(None, None)

# Градиентный бустинг

In [392]:
cat_features = [0, 6, 7]
data = train_dataframe.drop(['PassengerId', 'Survived'], axis=1)
labels = train_dataframe['Survived']
train_data, eval_data, train_labels, eval_labels = train_test_split(data, labels, test_size=0.2, random_state=42)

In [393]:
from catboost import CatBoostClassifier

model_cb = CatBoostClassifier(iterations=100,
                           learning_rate=0.1,
                           depth=5,
                           task_type="GPU",
                           devices='0:1')
model_cb.fit(train_data, train_labels, cat_features, verbose=False)

train_preds = model_cb.predict(train_data)
train_acc = accuracy_score(train_preds, train_labels)

eval_preds = model_cb.predict(eval_data)
eval_acc = accuracy_score(eval_preds,eval_labels)

print('Train Accuracy:', train_acc, 'Eval Accuracy:', eval_acc)

Train Accuracy: 0.8677918424753868 Eval Accuracy: 0.8258426966292135


**Для последующих моделей нужно разбить некоторые columns на one-hot-encoding, что мы и сделаем**

In [394]:
train_dataframe_ohe = pd.get_dummies(train_dataframe, prefix=['Pclass', 'Embarked', 'Cabin_Letter'])*1
test_dataframe_ohe = pd.get_dummies(test_dataframe, prefix=['Pclass', 'Embarked', 'Cabin_Letter'])*1
train_dataframe_ohe.head()

,PassengerId,Survived,Sex,Age,SibSp,Parch,Fare,Cabin_Num,Pclass_1,Pclass_2,...,Embarked_S,Cabin_Letter_A,Cabin_Letter_B,Cabin_Letter_C,Cabin_Letter_D,Cabin_Letter_E,Cabin_Letter_F,Cabin_Letter_G,Cabin_Letter_N,Cabin_Letter_T
0,1,0,1,-0.592148,1,0,-0.502163,-0.421085,0,0,...,1,0,0,0,0,0,0,0,1,0
1,2,1,0,0.638430,1,0,0.786404,2.737053,1,0,...,0,0,0,1,0,0,0,0,0,0
2,3,1,0,-0.284503,0,0,-0.488580,-0.421085,0,0,...,1,0,0,0,0,0,0,0,1,0
3,4,1,0,0.407697,1,0,0.420494,4.148926,1,0,...,1,0,0,1,0,0,0,0,0,0
4,5,0,1,0.407697,0,0,-0.486064,-0.421085,0,0,...,1,0,0,0,0,0,0,0,1,0


In [395]:
train_dataframe_ohe['Survived'].value_counts()

Survived
0    549
1    340
Name: count, dtype: int64

In [396]:
data = train_dataframe_ohe.drop(['PassengerId', 'Survived'], axis=1)
labels = train_dataframe_ohe['Survived']
train_data, eval_data, train_labels, eval_labels = train_test_split(data, labels, test_size=0.2, random_state=42)

# Логистическая регрессия

Сначала прогоним модель с l0-регуляризацией, чтобы определить, какие из фичей для нас не важны и от которых можно избавиться.

In [397]:
from sklearn.linear_model import LogisticRegression
model_log = LogisticRegression(penalty='l1',random_state=42, solver='liblinear')
model_log.fit(train_data, train_labels)

train_preds = model_log.predict(train_data)
train_acc = accuracy_score(train_preds, train_labels)

eval_preds = model_log.predict(eval_data)
eval_acc = accuracy_score(eval_preds,eval_labels)

print('Train Accuracy:', train_acc, 'Eval Accuracy:', eval_acc)
for i in range(len(train_data.columns)):
    print(train_data.columns[i], 'coef:', model_log.coef_[0][i])

Train Accuracy: 0.8227848101265823 Eval Accuracy: 0.8033707865168539
Sex coef: -2.6632340774153302
Age coef: -0.5594134809089262
SibSp coef: -0.3658159394101392
Parch coef: -0.07030566912861955
Fare coef: 0.12405752605869783
Cabin_Num coef: -0.04186008212804646
Pclass_1 coef: 1.4204297492369788
Pclass_2 coef: 1.1980549172809514
Pclass_3 coef: 0.0
Embarked_C coef: 0.5863713910721492
Embarked_Q coef: 0.38755850415650445
Embarked_S coef: 0.0
Cabin_Letter_A coef: 0.0
Cabin_Letter_B coef: 0.0
Cabin_Letter_C coef: 0.0
Cabin_Letter_D coef: 0.40044826087120966
Cabin_Letter_E coef: 0.8713651969068154
Cabin_Letter_F coef: 0.0
Cabin_Letter_G coef: 0.0
Cabin_Letter_N coef: -0.7605637099112743
Cabin_Letter_T coef: 0.0


In [398]:
data = train_dataframe_ohe.drop(['PassengerId', 'Survived', 'Embarked_S', 'Pclass_3', 'Cabin_Letter_A', 'Cabin_Letter_B', 'Cabin_Letter_C', 'Cabin_Letter_F', 'Cabin_Letter_G', 'Cabin_Letter_T'], axis=1)
labels = train_dataframe_ohe['Survived']
train_data, eval_data, train_labels, eval_labels = train_test_split(data, labels, test_size=0.2, random_state=42)

**Теперь можно построить очищенную модель логистической регрессии.**

In [399]:
model_log = LogisticRegression()
model_log.fit(train_data, train_labels)

train_preds = model_log.predict(train_data)
train_acc = accuracy_score(train_preds, train_labels)

eval_preds = model_log.predict(eval_data)
eval_acc = accuracy_score(eval_preds,eval_labels)

print('Train Accuracy:', train_acc, 'Eval Accuracy:', eval_acc)

Train Accuracy: 0.8213783403656821 Eval Accuracy: 0.8202247191011236


# SVM

Построим SVM на очищенных данных, из которых мы удалили ненужные фичи.

In [400]:
from sklearn.svm import SVC

model_svm = SVC(probability=True)
model_svm.fit(train_data, train_labels)

train_preds = model_svm.predict(train_data)
train_acc = accuracy_score(train_preds, train_labels)

eval_preds = model_svm.predict(eval_data)
eval_acc = accuracy_score(eval_preds,eval_labels)

print('Train Accuracy:', train_acc, 'Eval Accuracy:', eval_acc)

Train Accuracy: 0.8523206751054853 Eval Accuracy: 0.8258426966292135


# Random Forest

In [430]:
from sklearn.ensemble import RandomForestClassifier

model_rfc = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=5)
model_rfc.fit(train_data, train_labels)

train_preds = model_rfc.predict(train_data)
train_acc = accuracy_score(train_preds, train_labels)

eval_preds = model_rfc.predict(eval_data)
eval_acc = accuracy_score(eval_preds,eval_labels)

print('Train Accuracy:', train_acc, 'Eval Accuracy:', eval_acc)

Train Accuracy: 0.8677918424753868 Eval Accuracy: 0.8146067415730337


**Перестроим Catboost на очищенных данных, чтобы использовать его в ансамблировании. Как видим, удаление фичей не сказалось на качестве для Catboost-a.**

In [412]:
from catboost import CatBoostClassifier

model_cb = CatBoostClassifier(iterations=100,
                           learning_rate=0.1,
                           depth=5,
                           task_type="GPU",
                           devices='0:1')
model_cb.fit(train_data, train_labels, verbose=False)

train_preds = model_cb.predict(train_data)
train_acc = accuracy_score(train_preds, train_labels)

eval_preds = model_cb.predict(eval_data)
eval_acc = accuracy_score(eval_preds,eval_labels)

print('Train Accuracy:', train_acc, 'Eval Accuracy:', eval_acc)

Train Accuracy: 0.8804500703234881 Eval Accuracy: 0.8258426966292135


# Стеккинг

Я буду реализовать два варианта стеккинга: первый - из выходов вероятностей моделей будет браться среднее; второй - на выходах вероятностей моделей будет строиться новая линейная модель, т.е. выходы моделей будут браться с определенными коэффициентами.

In [402]:
class StackingMean:
    def __init__(self, models):
        self.models = models
    def predict_proba(self, X):
        predictions = []
        for model in models:
            proba = model.predict_proba(X)[:,1]
            predictions.append(proba)
        predictions = np.array(predictions)
        return predictions.mean(axis=0)
    def predict(self, X):
        proba = self.predict_proba(X)
        return (proba >= 0.5) * 1

In [403]:
models = [model_cb, model_log, model_svm]
stacking_mean_model = StackingMean(models)

train_preds = stacking_mean_model.predict(train_data)
train_acc = accuracy_score(train_preds, train_labels)

eval_preds = stacking_mean_model.predict(eval_data)
eval_acc = accuracy_score(eval_preds,eval_labels)

print('Train Accuracy:', train_acc, 'Eval Accuracy:', eval_acc)

Train Accuracy: 0.8523206751054853 Eval Accuracy: 0.8258426966292135


In [404]:
class StackingLinear:
    def __init__(self):
        self.linear = LogisticRegression()
    
    def fit(self, X, models, Y):
        predictions = []
        self.models = models
        for model in models:
            proba = model.predict_proba(X)[:,1]
            predictions.append(proba)
        predictions = np.array(predictions)
        predictions = predictions.transpose()
        self.linear.fit(predictions, Y)
        
    def predict_proba(self, X):
        predictions = []
        for model in self.models:
            proba = model.predict_proba(X)[:,1]
            predictions.append(proba)
        predictions = np.array(predictions)
        predictions = predictions.transpose()
        preds = self.linear.predict_proba(predictions)
        return preds[:, 1]
    
    def predict(self, X):
        proba = self.predict_proba(X)
        return (proba >= 0.5) * 1

In [405]:
models = [model_cb, model_log, model_svm]
stacking_linear_model = StackingLinear()
stacking_linear_model.fit(train_data, models, train_labels)

train_preds = stacking_linear_model.predict(train_data)
train_acc = accuracy_score(train_preds, train_labels)

eval_preds = stacking_linear_model.predict(eval_data)
eval_acc = accuracy_score(eval_preds,eval_labels)

print('Train Accuracy:', train_acc, 'Eval Accuracy:', eval_acc)

Train Accuracy: 0.8762306610407876 Eval Accuracy: 0.8089887640449438


# Беггинг

Для беггинга аналогично стеккингу будут Mean и Linear.

In [406]:
class BaggingMean:
    def __init__(self, models):
        self.models = models
        self.stacking_mean = StackingMean(models)
    
    def fit(self, X, Y):
        batches = []
        labels = []
        batch_size = len(X) // len(self.models)
        
        for begin in range(0, len(X), batch_size):
            end = min(begin + batch_size, len(X))
            batches.append(X[begin:end])
            labels.append(Y[begin:end])
            
        for i in range(len(self.models)):
            temp_X = []
            temp_Y = []
            for j in range(len(batches)):
                if i != j:
                    temp_X += batches[j]
                    temp_Y += labels[j]
            self.models[i].fit(temp_X, temp_Y)
    
    def predict_proba(self, X):
        return self.stacking_mean.predict_proba(X)
    
    def predict(self, X):
        return self.stacking_mean.predict(X)

In [407]:
models = [LogisticRegression(), LogisticRegression(), LogisticRegression(), LogisticRegression(), LogisticRegression()]
bagging_mean_model = BaggingMean(models)
bagging_mean_model.fit(train_data.values.tolist(), train_labels.values.tolist())

train_preds = bagging_mean_model.predict(train_data)
train_acc = accuracy_score(train_preds, train_labels)

eval_preds = bagging_mean_model.predict(eval_data)
eval_acc = accuracy_score(eval_preds,eval_labels)

print('Train Accuracy:', train_acc, 'Eval Accuracy:', eval_acc)

Train Accuracy: 0.819971870604782 Eval Accuracy: 0.8146067415730337


/opt/conda/lib/python3.10/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warning

In [408]:
class BaggingLinear:
    def __init__(self, models):
        self.models = models
        self.stacking_linear = StackingLinear()
    
    def fit(self, X, Y):
        batches = []
        labels = []
        batch_size = len(X) // len(self.models)
        
        for begin in range(0, len(X), batch_size):
            end = min(begin + batch_size, len(X))
            batches.append(X[begin:end])
            labels.append(Y[begin:end])
            
        for i in range(len(self.models)):
            temp_X = []
            temp_Y = []
            for j in range(len(batches)):
                if i != j:
                    temp_X += batches[j]
                    temp_Y += labels[j]
            self.models[i].fit(temp_X, temp_Y)
            
        self.stacking_linear.fit(X, self.models, Y)
    
    def predict_proba(self, X):
        return self.stacking_linear.predict_proba(X)
    
    def predict(self, X):
        return self.stacking_linear.predict(X)

In [439]:
models = [LogisticRegression(), SVC(probability=True), SVC(probability=True), LogisticRegression(), SVC(probability=True)]
bagging_linear_model = BaggingLinear(models)
bagging_linear_model.fit(train_data.values.tolist(), train_labels.values.tolist())

train_preds = bagging_linear_model.predict(train_data)
train_acc = accuracy_score(train_preds, train_labels)

eval_preds = bagging_linear_model.predict(eval_data)
eval_acc = accuracy_score(eval_preds,eval_labels)

print('Train Accuracy:', train_acc, 'Eval Accuracy:', eval_acc)

Train Accuracy: 0.8452883263009845 Eval Accuracy: 0.8258426966292135


/opt/conda/lib/python3.10/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but SVC was fitted without feature names
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but SVC was fitted without feature names
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but SVC was fitted without feature names
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
/opt/conda/lib/python3.10/site-packag

# Подготовка Submission

У меня при валидации лучше всего себя показали Catboost и SVM. Беггинг и стеккинг на моих данных особого выхлопа не дали. Однако на реальном датасете для проверки kaggle беггинг с использованием SVM и LogRegression дал наилучший результат.

In [410]:
models = [LogisticRegression(), SVC(probability=True), SVC(probability=True), LogisticRegression(), SVC(probability=True)]
final_model = BaggingMean(models)
final_model.fit(data.values.tolist(), labels.values.tolist())

test_data = test_dataframe_ohe.drop(['PassengerId', 'Embarked_S', 'Pclass_3', 'Cabin_Letter_A', 'Cabin_Letter_B', 'Cabin_Letter_C', 'Cabin_Letter_F', 'Cabin_Letter_G'], axis=1)

predictions = final_model.predict(test_data)

/opt/conda/lib/python3.10/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but SVC was fitted without feature names
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but SVC was fitted without feature names
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but SVC was fitted without feature names
  warnings.warn(


In [411]:
predict_dataframe = test_dataframe_ohe[['PassengerId']]
predict_dataframe['Survived'] = predictions
predict_dataframe.to_csv('/kaggle/working/submission.csv', index=False)

/tmp/ipykernel_34/3443672090.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predict_dataframe['Survived'] = predictions
